# $(\mathrm{H}_2\mathrm{O})_3$ cost estimate

In [1]:
import numpy as np
import openfermion
import pyscf
from pyscf import gto

/home/mitarai/anaconda/envs/qulacs/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
  h5py.get_config().default_file_mode = 'a'


In [2]:
xyz_file = "TIP4P-3.xyz"

Get energy gap of a single molecule

In [3]:
atom = [('O', [-0.3511375701099324, -2.220327851906733, 1.4480207843187745]), ('H', [-0.24292674995680064, -1.697798925312798, 3.1763661875721043]), ('H', [-0.11012983703262759, -0.699680546250837, 0.4985624750191387]),]
basis = 'sto3g'
mol = gto.M(atom=atom,
            basis=basis,
            unit="Bohr")
rhf = mol.RHF.run()
cisolver = pyscf.fci.FCI(rhf)
cisolver.nroots = 5
e0, e1, e2, e3, e4 = cisolver.kernel()[0]
Delta = e1-e0
p = cisolver.ci[0][0,0]**2

converged SCF energy = -74.9629282604051


Generate Hamiltonian

In [4]:
from local_hamiltonian import get_localized_hamiltonian, get_localized_integrals
basis = 'sto3g'
method = 'lowdin'
mol = gto.M(atom="TIP4P-3.xyz",
            basis=basis)
rhf = mol.RHF.run()
e_rhf = rhf.e_tot
of_hamiltonian = get_localized_hamiltonian(mol, method)
constant, one_body_int, two_body_int = get_localized_integrals(mol, method)

converged SCF energy = -224.911145040044
converged SCF energy = -224.911145040045
converged SCF energy = -224.911145040045


Split the Hamiltonian

In [5]:
groups = [list(range(14*i, 14*i+14)) for i in range(3)]
original_qubit_hamiltonian = openfermion.transforms.jordan_wigner(of_hamiltonian)
constant_in_original_qubit_hamiltonian = original_qubit_hamiltonian.terms[()]
original_qubit_hamiltonian -= constant_in_original_qubit_hamiltonian
# Note that original_qubit_hamiltonian should not have constant hereafter. 

In [6]:
from utils import local_group_qubit_operator
hamiltonian_list = []
for i in range(3):
    hamiltonian_list.append(local_group_qubit_operator(original_qubit_hamiltonian, groups[i]))
for i in range(3):
    hamiltonian_interaction = original_qubit_hamiltonian - hamiltonian_list[i]

Get the pauli coefficients

In [7]:
def coef2array(h):
    _array = np.array([h.terms[key] for key in h.terms])
    return _array[np.where(np.abs(_array)>1e-11)]
original_coef = coef2array(original_qubit_hamiltonian)
grouped_coef_list = []
for i in range(3):
    grouped_coef_list.append(coef2array(hamiltonian_list[i]))
interaction_coef = coef2array(hamiltonian_interaction)

Calculate norms

In [8]:
h1norm = abs(e_rhf-constant_in_original_qubit_hamiltonian)
for i in range(3):
    h1norm += np.sum(np.abs(grouped_coef_list[i]))
v1norm = np.sum(np.abs(interaction_coef))
v23norm = np.sum(np.abs(interaction_coef)**(2/3))**(3/2)
lvlh = len(interaction_coef)/(sum([len(grouped_coef_list[i]) for i in range(3)]))
print(h1norm)
print(v1norm)
print(v23norm)
print(lvlh)

323.1480500830674
197.99773994397404
10734.14976990793
27.620822241940253


# Generate values in the paper

In [9]:
from cost_estimate import get_cost_for_first_order_pert, get_cost_for_second_order_pert
delta0 = 0.3e-3
delta1 = 0.3e-3
delta2 = 0.3e-3
r_scale = 1/3
epsilon_scale = 1/3
n_subsystems = 3
print(get_cost_for_first_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))
print(get_cost_for_second_order_pert(h1norm, v23norm, v1norm, delta0, delta1, Delta, p, r_scale=r_scale, epsilon_scale=epsilon_scale, n_subsystems=n_subsystems))

epsilon:  1.534170028134632e-07
n_filter:  (142815.0282102293+0j)
kappa:  0.0012321826788653186
M1:  163198.89865177913
r:  2.5252813498855152e-08
x_th: 9.283670439072213e-07
(2577625933006.125+0j)
r:  5.08221771959738e-11
epsilon_filter:  3.087571253086123e-10
epsilon_ptb:  2.4728759359300236e-07
kappa:  0.0012321826788653186
w:  0.0012321826788653186
w0:  9.283670439072213e-07
M2:  29330331795682.418
n_filter:  (196931.68171834922+0j)
n_ptb:  (941380.9948579331+0j)
(8.844606209167905e+20+0j)
